In [ ]:
# Processor requirements
# The first two cells must be code cells, inpath and outpath define the scan file location and output location
inpath = '/dls/i16/data/2025/mm41580-1/processed/1114510_msmapper.nxs'
outpath = ""

# Plot msmapper HKL cuts

This notebook automatically loads remapped HKL volumes resulting from the MillerSpaceMapper software on I16 and plots cuts along the pricipal axes.

See https://confluence.diamond.ac.uk/display/I16/HKL+Mapping


Running MillSpaceMapper from command line:
```bash
module load msmapper
rs_map -s 0.002 -o /dls/i16/data/2022/mm12345-1/processing/12345_remap.nxs /dls/i16/data/2022/mm12345-1/12345.nxs
```

Running this notebook from command line:
```bash
module load python/3
python /dls_sw/i16/software/python/jupyter_processor/jproc.py mapper /dls/i16/data/2022/mm12345-1/12345.nxs
```

Using the autoprocessing in GDA on I16:
```python
from gda.device.scannable import ProcessingScannable
remapHKLcuts = ProcessingScannable('remapHKLcuts')
remapHKLcuts['msmapper'] = "{'notebook': '/dls_sw/i16/software/python/userscripts/i16user/jupyter_processors/msmapper_processor.ipynb'}"
scancn eta 0.001 101 remapHKLcuts pil 1  # results will appear in ispyb
```

In [ ]:
import sys, os
import glob, time
import numpy as np
import matplotlib.pyplot as plt

from mmg_toolbox import data_file_reader, module_info

print(module_info())

In [ ]:
sys.path.append("/dls_sw/i16/software/daqmessenger")
daq = None
try:
    from daqmessenger import DaqMessenger
    daq = DaqMessenger("i16-control")
    daq.connect()
    daq.send_file(inpath)
    daq.disconnect()
except ImportError:
    print("no messenger")




In [ ]:
# Get remapped file
print(inpath)
print(outpath)
scan = data_file_reader(inpath)
print(scan)

In [ ]:
# If scan is a standard nexus file, not a remapped file, look for the remapped file in the processing folder

if 'volume' in scan:
    remap = scan
    print('File contains remapped data')
else:
    # Find remapped files
    proc_dir = os.path.dirname(scan.filename) + '/processed/'
    for ntries in range(10):
        # remapping may take some time, so keep checking until finished
        files = [proc_dir + file for file in os.listdir(proc_dir)]
        scn = str(scan.scan_number())
        mapper_files = [
            file for file in files
            if file.endswith('.nxs') and scn in file and
               'volume' in data_file_reader(file).map
        ]
        if len(mapper_files) > 0:
            remap = data_file_reader(mapper_files[0])
            break
        else:
            print('Remapped file does not exist yet, try again in 1 min')
            time.sleep(60)

print('Remapped file loaded: %s' % remap.filename)

In [ ]:
# Get reciprocal space data from file
h = remap('h_axis')
k = remap('k_axis')
l = remap('l_axis')
vol = remap('volume')

print('\n'.join(remap.get_string(*['h_axis', 'k_axis', 'l_axis', 'volume'])))

In [ ]:
# Plot summed cuts
plt.figure(figsize=(18, 8), dpi=60)
title = scan.format('#{scan_number:.0f}: {(cmd|user_command|scan_command)}')
plt.suptitle(title, fontsize=18)

plt.subplot(131)
plt.plot(h, vol.sum(axis=1).sum(axis=1))
plt.xlabel('h-axis (r.l.u.)', fontsize=16)
plt.ylabel('sum axes [1,2]', fontsize=16)

plt.subplot(132)
plt.plot(k, vol.sum(axis=0).sum(axis=1))
plt.xlabel('k-axis (r.l.u.)', fontsize=16)
plt.ylabel('sum axes [0,2]', fontsize=16)

plt.subplot(133)
plt.plot(l, vol.sum(axis=0).sum(axis=0))
plt.xlabel('l-axis (r.l.u.)', fontsize=16)
plt.ylabel('sum axes [0,1]', fontsize=16)

In [ ]:
# Plot summed images
plt.figure(figsize=(18, 8), dpi=60)
title = scan.format('#{scan_number:.0f}: {(cmd|user_command|scan_command)}')
plt.suptitle(title, fontsize=20)
plt.subplots_adjust(wspace=0.3)

plt.subplot(131)
K, H = np.meshgrid(k, h)
plt.pcolormesh(H, K, vol.sum(axis=2))
plt.xlabel('h-axis (r.l.u.)', fontsize=16)
plt.ylabel('k-axis (r.l.u.)', fontsize=16)
#plt.axis('image')
#plt.colorbar()

plt.subplot(132)
L, H = np.meshgrid(l, h)
plt.pcolormesh(H, L, vol.sum(axis=1))
plt.xlabel('h-axis (r.l.u.)', fontsize=16)
plt.ylabel('l-axis (r.l.u.)', fontsize=16)
#plt.axis('image')
#plt.colorbar()

plt.subplot(133)
L, K = np.meshgrid(l, k)
plt.pcolormesh(K, L, vol.sum(axis=0))
plt.xlabel('k-axis (r.l.u.)', fontsize=16)
plt.ylabel('l-axis (r.l.u.)', fontsize=16)
#plt.axis('image')
plt.colorbar()

## Convert to cartesian coordinates (Q)


In [ ]:
from mmg_toolbox.diffraction.lattice import bmatrix, cal2theta

# Convert to Q
a, b, c, alpha, beta, gamma = scan('unit_cell')
energy = scan('incident_energy')

# Build reciprocal lattice in orthogonal basis
astar, bstar, cstar = bmatrix(a, b, c, alpha, beta, gamma)
kk, hh, ll = np.meshgrid(k, h, l)
q = astar * hh.reshape(-1, 1) + bstar * kk.reshape(-1, 1) + cstar * ll.reshape(-1, 1)
qx = q[:, 0].reshape(hh.shape)
qy = q[:, 1].reshape(hh.shape)
qz = q[:, 2].reshape(hh.shape)
# magnitude of wavevector-transfer Q=kf-ki
qmag = np.sqrt(qx **2 + qy ** 2 + qz **2)

In [ ]:
# Plot summed images
plt.figure(figsize=(18, 8), dpi=60)
title = scan.format('#{scan_number:.0f}: {(cmd|user_command|scan_command)}')
plt.suptitle(title, fontsize=20)
plt.subplots_adjust(wspace=0.3)

plt.subplot(131)
plt.pcolormesh(qx.mean(axis=2), qy.mean(axis=2), vol.sum(axis=2))
plt.xlabel('Qx [A$^{-1}$]', fontsize=16)
plt.ylabel('Qy [A$^{-1}$]', fontsize=16)
#plt.axis('image')
#plt.colorbar()

plt.subplot(132)
plt.pcolormesh(qx.mean(axis=1), qz.mean(axis=1), vol.sum(axis=1))
plt.xlabel('Qx [A$^{-1}$]', fontsize=16)
plt.ylabel('Qz [A$^{-1}$]', fontsize=16)
#plt.axis('image')
#plt.colorbar()

plt.subplot(133)
L, K = np.meshgrid(l, k)
plt.pcolormesh(qy.mean(axis=0), qz.mean(axis=0), vol.sum(axis=0))
plt.xlabel('Qy [A$^{-1}$]', fontsize=16)
plt.ylabel('Qz [A$^{-1}$]', fontsize=16)
#plt.axis('image')
plt.colorbar()

### |Q| and Two-Theta plots

In [ ]:

qmag = qmag.reshape(-1)
qvol = vol.reshape(-1)
bin_cen = np.arange(qmag.min(), qmag.max(), 0.001)
bin_edge = bin_cen + 0.005
bin_pos = np.digitize(qmag, bin_edge) -1
bin_sum = [np.mean(qvol[bin_pos==n]) for n in range(len(bin_cen))]
tth = cal2theta(bin_cen, energy)

title2 = scan.format('#{scan_number:.0f}\n{(cmd|user_command|scan_command)}')

plt.figure(dpi=100)
plt.plot(bin_cen, bin_sum)
plt.title(title2, fontsize=20)
plt.xlabel('|Q| A$^{-1}$', fontsize=20)
plt.ylabel('Intensity', fontsize=20)

plt.figure(dpi=100)
plt.plot(tth, bin_sum)
plt.title(title2, fontsize=20)
plt.xlabel('Two-Theta [Deg]', fontsize=20)
plt.ylabel('Intensity', fontsize=20)